In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

c:\Users\AMD\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
# Freeze base model
base_model.trainable = False

In [ ]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(4, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 4)                 2052      
                                                                 
Total params: 14716740 (56.14 MB)
Trainable params: 2052 (8.02 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [ ]:
model = keras.Model(inputs, outputs)
model.summary()
model.compile(loss = keras.losses.CategoricalCrossentropy(from_logits=True) , metrics = [keras.metrics.CategoricalAccuracy()])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 4)                 2052      
                                                                 
Total params: 14716740 (56.14 MB)
Trainable params: 2052 (8.02 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen_train = ImageDataGenerator(
    samplewise_center=True,  # set each sample mean to 0
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.1,  # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,)
datagen_valid = ImageDataGenerator(samplewise_center=True)

In [ ]:
# load and iterate training dataset
train_it = datagen_train.flow_from_directory(
    "C:\\Users\\AMD\\OneDrive\\Área de Trabalho\\PYTHON\\Detecção de Tumores Cerebrais por IA\\archive\\Data\\train",
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
)
# load and iterate validation dataset
valid_it = datagen_valid.flow_from_directory(
    "C:\\Users\\AMD\\OneDrive\\Área de Trabalho\\PYTHON\\Detecção de Tumores Cerebrais por IA\\archive\\Data\\test",
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
)

Found 2383 images belonging to 4 classes.
Found 713 images belonging to 4 classes.


In [ ]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=20)

Epoch 1/20


c:\Users\AMD\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\backend.py:5577: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


74/74 [==============================] - 624s 8s/step - loss: 1.8839 - categorical_accuracy: 0.4222 - val_loss: 1.6107 - val_categorical_accuracy: 0.5049
Epoch 2/20
74/74 [==============================] - 618s 8s/step - loss: 0.9647 - categorical_accuracy: 0.6504 - val_loss: 1.0086 - val_categorical_accuracy: 0.6662
Epoch 3/20
74/74 [==============================] - 656s 9s/step - loss: 0.7446 - categorical_accuracy: 0.7268 - val_loss: 0.9696 - val_categorical_accuracy: 0.6872
Epoch 4/20
74/74 [==============================] - 650s 9s/step - loss: 0.6497 - categorical_accuracy: 0.7768 - val_loss: 0.8185 - val_categorical_accuracy: 0.7167
Epoch 5/20
74/74 [==============================] - 643s 9s/step - loss: 0.6016 - categorical_accuracy: 0.7809 - val_loss: 0.7261 - val_categorical_accuracy: 0.7405
Epoch 6/20
74/74 [==============================] - 637s 9s/step - loss: 0.5542 - categorical_accuracy: 0.8011 - val_loss: 0.7592 - val_categorical_accuracy: 0.7293
Epoch 7/20
74/74 [===

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),  # Very low learning rate
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
model.fit(train_it, steps_per_epoch=37, validation_data=valid_it, validation_steps=4, epochs=10)

Epoch 1/10
37/37 [==============================] - 839s 23s/step - loss: 0.2459 - binary_accuracy: 0.9198 - val_loss: 0.1873 - val_binary_accuracy: 0.9375
Epoch 2/10
37/37 [==============================] - 829s 22s/step - loss: 0.2062 - binary_accuracy: 0.9348 - val_loss: 0.2486 - val_binary_accuracy: 0.9082
Epoch 3/10
37/37 [==============================] - 818s 22s/step - loss: 0.1781 - binary_accuracy: 0.9514 - val_loss: 0.2181 - val_binary_accuracy: 0.9160
Epoch 4/10
37/37 [==============================] - 823s 22s/step - loss: 0.1513 - binary_accuracy: 0.9544 - val_loss: 0.1449 - val_binary_accuracy: 0.9375
Epoch 5/10
37/37 [==============================] - 828s 22s/step - loss: 0.1323 - binary_accuracy: 0.9611 - val_loss: 0.3823 - val_binary_accuracy: 0.8887
Epoch 6/10
37/37 [==============================] - 818s 22s/step - loss: 0.1237 - binary_accuracy: 0.9674 - val_loss: 0.1386 - val_binary_accuracy: 0.9590
Epoch 7/10
37/37 [==============================] - 821s 23s/ste

In [ ]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

22/22 [==============================] - 149s 7s/step - loss: 0.1523 - binary_accuracy: 0.9520


[0.15228371322155, 0.9519635438919067]

In [ ]:
import os

# Caminho absoluto para o diretório de salvamento
save_dir = os.path.abspath('C:/Users/AMD/OneDrive')

# Crie o diretório se ele não existir
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

# Salve o modelo no caminho absoluto
model.save(save_dir)


INFO:tensorflow:Assets written to: C:\Users\AMD\OneDrive\assets


INFO:tensorflow:Assets written to: C:\Users\AMD\OneDrive\assets
